In [1]:
import os
import sys

root_path = os.path.abspath(os.path.join(os.getcwd(), '../../..'))
sys.path.append(root_path)

In [4]:
from quants_lab.controllers.market_making.dman_maker_v2 import DManMakerV2Config
from hummingbot.strategy_v2.backtesting import MarketMakingBacktesting
import traceback
from decimal import Decimal
import pandas_ta as ta  # noqa: F401
import pandas as pd
from optuna import TrialPruned

backtesting_engine = MarketMakingBacktesting()

async def objective(trial, start, end, backtesting_resolution):
    try:
        # Market configuration
        connector_name = "binance_perpetual"
        trading_pair = "WLD-USDT"

        # Account configuration
        total_amount_quote = 1000
        trade_cost = 0.0006
        buy_spreads = [Decimal("0.002")]
        sell_spreads = [Decimal("0.002")]
        buy_amounts_pct = [Decimal("1")]
        sell_amounts_pct = [Decimal("1")]
        executor_refresh_time = 60
        cooldown_time = 3600
        top_executor_refresh_time = 60
        executor_activation_bounds = [Decimal("0.01")]
        dca_spreads = [Decimal("0."), Decimal("0.002"), Decimal("0.004"), Decimal("0.006"), Decimal("0.008"), Decimal("0.01")]
        dca_amounts = [Decimal("0.1"), Decimal("0.2"), Decimal("0.3"), Decimal("0.4"), Decimal("0.5"), Decimal("0.6")]

        # Triple barrier configuration
        stop_loss = trial.suggest_float('stop_loss', 0.01, 0.04, step=0.01)
        take_profit = trial.suggest_float('take_profit', 0.01, 0.04, step=0.01)
        time_limit = 60 * 60 * 12  # 12 hours
        trailing_stop_activation_price_delta = Decimal("0.008")
        trailing_stop_trailing_delta = Decimal("0.004")
        config_dict = {
            "connector_name": connector_name,
            "trading_pair": trading_pair,
            "total_amount_quote": total_amount_quote,
            "buy_spreads": buy_spreads,
            "sell_spreads": sell_spreads,
            "buy_amounts_pct": buy_amounts_pct,
            "sell_amounts_pct": sell_amounts_pct,
            "executor_refresh_time": executor_refresh_time,
            "cooldown_time": cooldown_time,
            "top_executor_refresh_time": top_executor_refresh_time,
            "executor_activation_bounds": executor_activation_bounds,
            "stop_loss": stop_loss,
            "take_profit": take_profit,
            "time_limit": time_limit,
            "trailing_stop": {
                "activation_price": trailing_stop_activation_price_delta,
                "trailing_delta": trailing_stop_trailing_delta
            },
            "dca_spreads": dca_spreads,
            "dca_amounts": dca_amounts
        }


        config = DManMakerV2Config(**config_dict)
        start_time = pd.to_datetime(start).timestamp() * 1e3
        end_time = pd.to_datetime(end).timestamp() * 1e3
        backtesting_results = await backtesting_engine.run_backtesting(
                    controller_config=config, trade_cost=trade_cost,
                    start=int(start_time), end=int(end_time),
                    backtesting_resolution=backtesting_resolution)

        strategy_analysis = backtesting_results["results"]

        trial.set_user_attr("net_pnl_quote", strategy_analysis["net_pnl_quote"])
        trial.set_user_attr("net_pnl_pct", strategy_analysis["net_pnl"])
        trial.set_user_attr("max_drawdown_usd", strategy_analysis["max_drawdown_usd"])
        trial.set_user_attr("max_drawdown_pct", strategy_analysis["max_drawdown_pct"])
        trial.set_user_attr("sharpe_ratio", strategy_analysis["sharpe_ratio"])
        trial.set_user_attr("accuracy", strategy_analysis["accuracy"])
        trial.set_user_attr("total_positions", strategy_analysis["total_positions"])
        trial.set_user_attr("profit_factor", strategy_analysis["profit_factor"])
        trial.set_user_attr("win_signals", strategy_analysis["win_signals"])
        trial.set_user_attr("loss_signals", strategy_analysis["loss_signals"])
        trial.set_user_attr("close_types", strategy_analysis["close_types"])
        trial.set_user_attr("config", config_dict)
        return strategy_analysis["net_pnl"]
    except Exception as e:
        traceback.print_exc()
        raise TrialPruned()


In [6]:
import optuna

# Now let's configure the parameters for the optimization
study_name = "dman_maker_v2_opt"
storage= "sqlite:///../../../data/backtesting/backtesting_report.db"

study = optuna.create_study(direction="maximize", study_name=study_name,
                            storage=storage,
                            load_if_exists=True) # If the study already exists, we load it)

[I 2024-05-21 19:14:06,636] A new study created in RDB with name: dman_maker_v2_opt


In [7]:
# Not let's run the optimization!
start = "2024-05-01"
end = "2024-05-20"
backtesting_resolution = "1m"
n_trials = 200
study.optimize(lambda trial: objective(trial, start, end, backtesting_resolution), n_trials=n_trials)


[W 2024-05-21 19:14:32,364] Trial 0 failed with parameters: {} because of the following error: The value <coroutine object objective at 0x17fad5b60> could not be cast to float.
[W 2024-05-21 19:14:32,365] Trial 0 failed with value <coroutine object objective at 0x17fad5b60>.
/Users/dardonacci/anaconda3/envs/dashboard/lib/python3.10/site-packages/optuna/study/_optimize.py:159: RuntimeWarning: coroutine 'objective' was never awaited
  frozen_trial = _run_trial(study, func, catch)
[W 2024-05-21 19:14:32,380] Trial 1 failed with parameters: {} because of the following error: The value <coroutine object objective at 0x17fad5b60> could not be cast to float.
[W 2024-05-21 19:14:32,380] Trial 1 failed with value <coroutine object objective at 0x17fad5b60>.
[W 2024-05-21 19:14:32,390] Trial 2 failed with parameters: {} because of the following error: The value <coroutine object objective at 0x17fad5b60> could not be cast to float.
[W 2024-05-21 19:14:32,391] Trial 2 failed with value <coroutine